In [ ]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import numpy as np
import training
import config
import matplotlib.pyplot as plt
import FigureConfig as FC
import pandas
from scipy import optimize as op
import math

In [ ]:
data = pandas.read_csv('../result/Power_Neg_weight_data_10K_final.csv')
data

In [ ]:
y = data.to_numpy()
y = torch.tensor(y.astype(float)).T.abs()
y.shape

In [ ]:
x = pandas.read_csv('../result/Power_Neg_weight_data_10K_final.csv',nrows= 0)
x = x.columns.values.tolist()
result = []
for j in range(10000):
    a = x[j].split(' ')
    nums = []
    for k in range(2, 15, 2):
        
        nums.append(a[k])

    b=map(float,nums)    
    result.append(list(b))
x = torch.tensor(result)
x.shape

In [ ]:
x[:,0] = x[:,0]
x[:,1] = x[:,1]
x[:,2] = x[:,2]/1e3
x[:,3] = x[:,3]/1e3
x[:,4] = x[:,4]/1e3
x[:,5] = x[:,5]*1e6
x[:,6] = x[:,6]*1e6

In [ ]:
x.max(0)[0]

In [ ]:
x.min(0)[0]

# Visualization of neg power

In [ ]:
y.mean(1).max()

In [ ]:
torch.argmin((y.mean(1) - 0.01).abs())

In [ ]:
x1 = np.linspace(-2,2,4001)

inv_vis = [9990, 8396, 1000, 9004, 6407, 9533]

In [ ]:
inv_vis = [1185, 8396, 154, 50, 70, 6]

In [ ]:
for idx, i in enumerate(inv_vis):
    plt.plot(x1, y[i,:].mean()*torch.ones(x1.shape), lw=3, c=FC.liste[idx])
    plt.xticks([-2, -1, 0, 1, 2], ['-2', '-1', '0', '1', '2'], fontsize=15)
    plt.yticks([0.0, 0.01, 0.02, 0.03], ['0', '10', '20', '30'], fontsize=15)
    plt.xlim([-2,2])
    # plt.ylim([0,0.3])
    plt.axis('off')
plt.savefig(f'../../figures/inv_power_example.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
yy = torch.tanh(torch.tensor(x1)*20+1)*0.000005 + 0.004445

In [ ]:
plt.figure(figsize=[9,3])
plt.plot(x1, yy, lw=7, c=FC.liste[1])
plt.xticks([-2, -1, 0, 1, 2], ['-2', '-1', '0', '1', '2'], fontsize=15)
# plt.yticks([0, 0.1, 0.2, 0.3], ['0', '0.1', '0.2', '0.3'], fontsize=15)
plt.xlim([-2,2])
plt.axis('off')
plt.savefig(f'../../figures/inv_one_example_legend.pdf', format='pdf', bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
plt.plot(x1, y[1,:], lw=3, c=FC.liste[1])
plt.plot(x1, yy, lw=3, c=FC.liste[2])
plt.xticks([-2, -1, 0, 1, 2], ['-2', '-1', '0', '1', '2'], fontsize=15)
# plt.yticks([0, 0.1, 0.2, 0.3], ['0', '0.1', '0.2', '0.3'], fontsize=15)
plt.xlim([-2,2])
# plt.ylim([0,0.3])
# plt.savefig(f'../../figures/inv_power_example.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
for idx, i in enumerate(inv_vis):
    # plt.plot(x1,y1,'--',c='gray')
    # plt.plot(x2,y2,'--',c='gray')
    plt.plot(x1, y[i,:], lw=3, c=FC.liste[idx], label=f'{x[i,:]}')
    plt.xticks([-2, -1, 0, 1, 2], ['-2', '-1', '0', '1', '2'], fontsize=15)
    plt.yticks([0.0, 0.01, 0.02, 0.03], ['0', '10', '20', '30'], fontsize=15)
    plt.xlim([-2,2])
    # plt.ylim([0,0.3])
plt.legend()
plt.savefig(f'../../figures/inv_power_example_legend.pdf', format='pdf', bbox_inches='tight')
plt.show()

# Visualization for NN

In [ ]:
with open('./data/inv_power_dataset.p', 'rb') as file:
    a = pickle.load(file)

In [ ]:
X, Y = a['X'], a['Y']
Xn, Yn = a['Xn'], a['Yn']
X_min, X_max = a['X_min'], a['X_max']
Y_min, Y_max = a['Y_min'], a['Y_max']

X_learn, Y_learn = a['X_learn'], a['Y_learn']
X_train, Y_train = a['X_train'], a['Y_train']
X_valid, Y_valid = a['X_valid'], a['Y_valid']
X_test , Y_test  = a['X_test'] , a['Y_test']

Xn_learn, Yn_learn = a['Xn_learn'], a['Yn_learn']
Xn_train, Yn_train = a['Xn_train'], a['Yn_train']
Xn_valid, Yn_valid = a['Xn_valid'], a['Yn_valid']
Xn_test , Yn_test  = a['Xn_test'] , a['Yn_test']

In [ ]:
Yn_learn.max()

In [ ]:
eta_estimator = torch.load(f'model.negpow')

In [ ]:
prediction_train = eta_estimator(Xn_train)
prediction_valid = eta_estimator(Xn_valid)
prediction_test = eta_estimator(Xn_test)

lossfunction = torch.nn.MSELoss()
loss_train = lossfunction(Yn_train, prediction_train)
loss_valid = lossfunction(Yn_valid, prediction_valid)
loss_test  = lossfunction(Yn_test,  prediction_test)

plt.figure()
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100),  c='gray', alpha=0.5)
plt.scatter(Yn_train.flatten().numpy(), prediction_train.detach().flatten().numpy(), s=2, color=FC.Blue, label=f'train_loss: {loss_train:.2e}')
plt.scatter(Yn_valid.flatten()[::10].numpy(), prediction_valid.detach()[::10].flatten().numpy(), s=2, color=FC.Green, label=f'valid_loss: {loss_valid:.2e}')
plt.scatter(Yn_test[::20].flatten().numpy(), prediction_test[::20].detach().flatten().numpy(), s=2, color=FC.Pink, label=f'test_loss: {loss_test:.2e}')
plt.xlim([0, 1])
plt.ylim([0, 1])
ax = plt.gca()
ax.set_aspect(1)
plt.xticks([0, 0.25, 0.5, 0.75, 1], ['0', '0.25', '0.5', '0.75', '1'], fontsize=15)
plt.yticks([0, 0.25, 0.5, 0.75, 1], ['0', '0.25', '0.5', '0.75', '1'], fontsize=15)
# plt.legend(fontsize=12)
plt.axis('off')
plt.savefig(f'../../figures/final_INVPOW_result.pdf', format='pdf', bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
plt.figure()
plt.scatter(Yn_train.flatten().numpy(), prediction_train.detach().flatten().numpy()*0-1, s=30, color=FC.Blue, label=f'train data')
plt.scatter(Yn_valid.flatten().numpy(), prediction_valid.detach().flatten().numpy()*0-1, s=30, color=FC.Green, label=f'valid data')
plt.scatter(Yn_test.flatten().numpy(), prediction_test.detach().flatten().numpy()*0-1, s=30, color=FC.Pink, label=f'test data')
plt.xlim([0, 1])
plt.ylim([0, 1])
ax = plt.gca()
ax.set_aspect(1)
plt.legend(fontsize=12)
plt.savefig(f'../../figures/final_INV_legend.pdf', format='pdf', bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
loss_train, loss_valid, loss_test